In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH21-AGENT")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH21-AGENT


# Human-in-the-Loop

- `iter()` : 에이전트의 실행 과정을 단계별로 반복할 수 있게 해주는 interator 생성 
- Human-in-the-Loop : 중간 과정에서 사용자의 입력을 받아 계속 진행할지를 묻는 기능 제공 

In [3]:
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [4]:
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together.
    """
    return a + b

In [5]:
# 도구 정의
tools = [add_function]

# LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# prompt 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Agent 생성
gpt_agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=False,
    max_iterations=10,
    handle_parsing_errors=True,
)

In [6]:
# 질의 내용
question = "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"

In [7]:
# agent_executor를 반복적으로 실행
for step in agent_executor.iter({"input": question}):
    if output := step.get("intermediate_step"):
        action, value = output[0]
        if action.tool == "add_function":
            # Tool 실행 결과 출력
            print(f"\nTool Name: {action.tool}, 실행 결과: {value}")

        # 사용자에게 계속 진행할지 묻습니다.
        _continue = input("계속 진행하시겠습니다? (y/n)?:\n") or "Y"
        
        # 사용자가 'y'가 아닌 다른 입력을 하면 반복 중단
        if _continue.lower() != "y":
            break

# 최종 결과 출력
if "output" in step:
    print(step["output"])


Tool Name: add_function, 실행 결과: 235.7

Tool Name: add_function, 실행 결과: 380.0
114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.


--------
** End of Documents **